내용 : 나라에 따른 시간, 날짜를 구하고 시차를 계산하도록 하는 파트

주요점 : tool을 사용하는 방식이 어떻게 흘러가는지 이해하기

In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
# 🔄 기존 작성된 함수 사용
load_dotenv()
client=OpenAI() 

from datetime import datetime
import pytz

In [9]:
# 시간 구하는 함수
def get_current_time_tz(timezone="Asia/Seoul"):
  tz = pytz.timezone(timezone) 
  now = datetime.now(tz).strftime('%H:%M:%S')
  print(f"log : 현재 시간 : {now} {tz}")
  return now

# 날짜 구하는 함수
def get_current_date_tz(timezone="Asia/Seoul"):  
  tz = pytz.timezone(timezone) 
  now = datetime.now(tz).strftime('%Y 년 %m 월 %d 일') # 위 함수와 동일하지만 출력 형식만 날짜로 다름
  print(f"log : 현재 날짜 : {now} {tz}")
  return now

In [10]:
# tools 
# function을 사용하면 LLM이 한 가지 기능만 정의 but, tool은 여러 함수 묶어서 관리, 여러 함수 호출 가능
# LLM이 스스로 판단해서 불러올 함수를 정의하는거임 우리가 설정하는거는 description으로 추천하는 정도
tools = [
    {
    "type": "function", # 함수형 도구(함수인걸 지칭)
    "function": { 
                "name": "get_current_time_tz", # 불러올 함수 이름
                "description": "현재 시간 출력 HH:MM:SS format", # 이 함수를 언제 써야할지 판단시키기
                "parameters": { # 함수의 인자를 정의
                    "type": "object",  
                    "properties": {
                        "timezone": {
                            "type": "string",
                            "description": "Time zone in 'Area/Location' format, e.g., 'Asia/Seoul', 'America/New_York'. Default is 'Asia/Seoul'."
                            # LLM이 이해할 수 있도록 속성 설명
                        }
                    },
                    "required": ['timezone'] # 필수 입력값 지정
                    # get_current_time_tz를 실행할 때 무조건 timezone이란 키를 사용해야한다는 뜻
                }
            },
    },
    { 
        "type": "function", # 함수형 도구(함수인걸 지칭)
        "function": {
        "name": "get_current_date_tz", # 불러올 함수 이름
        "description": "현재 날짜 출력 YYYY 년 MM 월 DD 일 format", # 이 함수를 언제 써야할지 판단시키기
        "parameters": { # 함수의 인자를 정의
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "Time zone in 'Area/Location' format, e.g., 'Asia/Seoul', 'America/New_York'. Default is 'Asia/Seoul'."
                     # LLM이 이해할 수 있도록 속성 설명
                }
            },
            "required": ['timezone'] # 필수 입력값 지정
            # get_current_date_tz를 실행할 때 무조건 timezone이란 키를 사용해야한다는 뜻
        }}
    }
]

In [11]:
# 처음으로 gpt작동 => gpt에게 어떤 함수 사용할지 정하게하기
def get_first_response_tools(question): # 유저 입력값받기
    response = client.chat.completions.create( # 챗봇 생성
        model="gpt-4.1-mini", # 모델 지정
        messages=[
            {"role": "system", "content": "You are a helpful assistant. using locale language."}, # ai역할, 할일 지정
            {"role": "user", "content": question} # 유저역할, 입력값(변수)지정
        ],
        tools=tools, # 위의 tools를 사용해서 답변하도록 설정
        tool_choice="auto" # 어떤 함수를 사용할지 스스로 결정하게함(이제 ai가 알아서 tool안에 변수 골라서 사용)
    )
    return response

In [12]:
# response = get_first_response_tools(question)
# print(type(response))
# tool_calls = response.choices[0].message.model_dump_json(indent=2)
# print(tool_calls)
# ↪ 위의 코드를 실행한 (함수 get_first_response_tools를 실행한)결과

# {
#   "content": null,
#   "refusal": null,
#   "role": "assistant",
#   "annotations": [],
#   "audio": null,
#   "function_call": null,
#   "tool_calls": [ <= 여기 tool_calls가 중요 tool로 만들었으니 tool로 call한거(func에선 funcion_call이였음)
#     {
#       "id": "call_NDmFj30kZHsJ6Xx7Xtn0nn93",
#       "function": {
#         "arguments": "{\"timezone\": \"Asia/Seoul\"}", <= 함수에 전달해줄 파라미터 값
#         "name": "get_current_time_tz" <= 사용할 함수 스스로 지정
#       },
#       "type": "function"
#     },
#     {
#       "id": "call_LgHk12JSVqOt1PdVol685YGO",
#       "function": {
#         "arguments": "{\"timezone\": \"America/New_York\"}",
#         "name": "get_current_time_tz"
#       },
#       "type": "function"
#     }
#   ]
# }

In [ ]:
# 2번째 gpt 작동 => gpt에게 최종 답변 내놓게하기
# 이 코드 보기전에 바로 밑에 get_ai_response_tools 보고 오기
def get_followup_response_tools(question,tool_results):
    # tool_results는 get_ai_response_tools에서 정의해줌(tool_result는 함수 실행 결과 저장값)
    result_text = "\n".join([f"{t['name']} 결과는 {t['result']} 입니다." for t in tool_results])
    # tool_results는 함수를 실행한 결과값 모음
    # name = 실행할 함수명
    # result는 함수를 실행한 결과값
    followup_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. using locale language."},
            {"role": "user", "content": f"{question} 에 대해 다음 결과를 이용해 자연스러운 최종 답변을 만들어줘:\n{result_text}"}
        ],
        tools=tools
    )
    return followup_response.choices[0].message.content 

In [ ]:
# GPT가 호출한 함수를 실제 파이썬에서 실행 => 다시 GPT에게 최종응답 내보내는 함수
def get_ai_response_tools(question):
    response = get_first_response_tools(question=question) # 여기서 어떤 함수 쓸지 정함
    fn_name = getattr(response.choices[0].message.function_call, "name", None) # 함수 이름(함수명) 저장
    tool_results = [] #빈 배열 만들기
    for tool in response.choices[0].message.tool_calls: # for문으로 하나씩 넣어줌
        fn_name = tool.function.name # 사용할 함수명
        args = json.loads(tool.function.arguments) # 함수가 사용할 파라미터값(인자) 
        
        if fn_name:
            # 밑에 코드가 핵심! 실제로 파이썬에서 실행시키기 위해 있는 함수이기에 가장 중요!
            result = globals()[fn_name](**args) # 파일 내 정의된 함수를 인자를 활용하여 실행 => result에 저장
            tool_results.append({"name": fn_name, "result": result}) # 함수 실행 결과값 저장용
        else:
            tool_results.append({"name": fn_name, "result": f"Unknown function: {fn_name}"})
    final_response = get_followup_response_tools(question, tool_results) # 최종답변을 생성해주는 함수 실행
    return final_response

In [ ]:
question = '뉴욕과 시카고의 시차를 알려줘'
response = get_ai_response_tools(question) # 여기서 받은 question은 get_ai_response_tools 거쳐서 바로
# get_first_response_tools안으로 들어감
print(f'💬AI : {response}')

log : 현재 시간 : 01:48:15 America/New_York
log : 현재 시간 : 00:48:15 America/Chicago
💬AI : 뉴욕과 시카고의 현재 시차는 1시간입니다. 뉴욕이 01시 48분 15초일 때, 시카고는 00시 48분 15초로 시카고가 뉴욕보다 1시간 늦습니다.


In [ ]:
# 가장 보기좋은 실행 순서로는

# get_ai_response_tools(question)(유저가 입력) 

# => get_first_response_tools(question)(어떤 함수를 사용해 답변할지 결정)

# => get_ai_response_tools(question)(gpt가 결정한 함수를 실제로 실행) 

# => get_followup_response_tools(question, tool_results)(최종 답변을 gpt가 결정) 

# => get_ai_response_tools(question)(gpt가 결정한 최종 답변을 유저에게 보여줌)